In [3]:
import numpy as np
import os
import cv2
import keras
import glob
from keras.utils import np_utils
from keras.layers import Input, Dense, concatenate, Flatten, Dropout, multiply, BatchNormalization, Activation, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model, load_model
from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.initializers import TruncatedNormal, he_normal
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from random import shuffle

In [2]:
pwd

'/Users/vyettella/Desktop/not-so-random-forest/data'

In [5]:
class_folders = os.listdir('./classification_training_data/')
class_folders

['date_palm',
 'oak',
 'weeping_wattle',
 'white_mulberry',
 '.DS_Store',
 'carob',
 'golden_rain_tree',
 'common_myrtle',
 'poplar',
 'privet',
 'laurel',
 'black_locust',
 'pine',
 'oriental_plane',
 'eucalyptus',
 'persian_lilac',
 'judas',
 'tree_of_heaven',
 'mediterranean_cypress',
 'hibiscus',
 'olive',
 'silky_oak',
 'weeping_willow',
 'japanese_pagoda']

In [ ]:
willow_folder = "willow/*.jpg"
autumn_folder = "autumn/*.jpg"

files_willow = glob.glob(willow_folder) #label=0
files_autumn = glob.glob(autumn_folder)
print("Class 0: {} | Class 1: {}".format(len(files_willow), len(files_autumn)))
dataset = []

for fname in files_willow:
    dataset.append({"x": fname, "y": 0})
    
for i, fname in enumerate(files_autumn):
    dataset.append({"x": fname, "y": 1})
    if i == len(files_willow):
        break
        
indexes = [i for i in range(len(dataset))]
shuffle(indexes)
split_idx = int(len(indexes) * 0.8)
train_idx = indexes[0:split_idx]
val_idx = indexes[split_idx::]

print("Size Training set: {} | Valset {}".format(len(train_idx), len(val_idx)))


def generator_xy(batch_sz, dataset, index_set, is_training):
    idx = 0 # 'pointer for indexes
    while 1:
        if is_training:
            shuffle(index_set)
        x_batch = np.zeros( (batch_sz, 224, 224, 3), dtype=np.float32)
        y_batch = np.zeros( (batch_sz, 2), dtype=np.float32)
        for row in range(batch_sz):
            example = dataset[indexes[idx]]
            img_p = example["x"]
            y = example["y"]
            bgr = cv2.imread(img_p)
            bgr = cv2.resize(bgr, (224, 224))
            rgb = bgr[:,:,::-1]/255.
            x_batch[row] = rgb[:,:,:]
            y_batch[row, y] = 1
            idx += 1
            if (idx > len(indexes)-1):
                shuffle(indexes)
                idx = 0

        yield x_batch, y_batch